# 2021-06-23第5章面试中的算法

**漫画算法 小灰的算法之旅**
**作者：魏梦舒**
**Date:** 2021-06-23 20：29


**Author:** chenfengyuan
**E-mail:** chenfengyuan@cqu.edu.cn
**Goal:** 面试中的一些算法。

**更新记录**：

1. 2021-06-23 [x] 5.2 如何判断链表有环
2. 2021-06-24 [x] 5.3+5.4 最小栈的实现+如何求出最小公约数
3. 2021-06-27 [] 5.5+

## 5.1 踌躇满志的小灰

面试遇到的算法题目千变万化，不但要依靠扎实的算法基础，还需要随机应变。

即使面试“挂”掉也不必沮丧，就当是对自己的历练了。

## 5.2 如何判断链表有环

题目：有一个单向链表，链表中有可能出现“环”，那么如何用程序来判断该链表是否为有环链表呢？

***结题思路：***

**方法1：**首先从头节点开始，依次遍历单个链表中的每一个节点。每遍历一个新节点，就从头检查新节点之前的所有节点，用新节点和此节点之前所有节点依次作比较。如果发现新节点和之前的某个节点相同，则说明该节点被遍历过两次，链表有环；如果之前的所有节点中不存在与新节点相同的节点，就继续遍历下一个新节点，继续重复刚才的操作。

复杂度分析：假设链表的节点数量为n，则该解法的时间复杂度为$O(n^2)$,由于并没有创建额外的存储空间，所以空间复杂度为$O(1)$.

**方法2**：首先创建一个以节点ID为key的set集合，用来存储曾经遍历过的节点。然后同样从头节点开始，依次遍历单个链表中的每个节点。每遍历一个新节点，都用新节点和set集合中存储的节点进行比较，如果发现set中存在与之相同的节点ID，则说明链表有环，如果set中不存在与新节点相同的节点ID，就把这个新节点ID存入set中，之后进入下一个节点，继续重复刚才的操作。

复杂度分析：假设链表的节点数量为n，则该解法的时间复杂度为$O(n)$，由于使用了额外的存储空间，所以算法的空间复杂度为$O(n)$.

**方法3**：使用2个指针

首先创建两个指针p1和p2（在Python里就是两个对象的引用），让它们同时指向这个链表的头节点。然后开始一个大循环，在循环体中，让指针p1每次向后移动1个节点，让指针p2每次向后移动2个节点，然后比较两个指针指向的节点是否相同。如果相同，则可以判断出链表有环，如果不同，则继续下一次循环。

此方法类似于**追及问题**。假设链表的节点数量是n，则该算法的时间复杂度为$O(n)$.除两个指针外，没有使用额外的存储空间，所以空间复杂度是$O(1)$.


In [1]:
# 方法3的代码实现

class Node:
    def __init__(self, data):
        self.data = data
        self.next = None

def is_cycle(head):
    p1 = head
    p2 = head
    while p2 is not None and p2.next is not None:
        p1 = p1.next
        p2 = p2.next.next
        if p1 == p2:
            return True
    return False


node1 = Node(5)
node2 = Node(3)
node3 = Node(7)
node4 = Node(2)
node5 = Node(6)

node1.next = node2
node2.next = node3
node3.next = node4
node4.next = node5
node5.next = node2
print(is_cycle(node1))

True


***问题拓展***：
1. 如果链表有环，如何求出环的长度？
2. 如何求出入环节点？

Question1：当两个指针首次相遇，证明链表有环的时候，让两个指针从相遇点继续循环前进，并统计前进的循环次数，直到两个指针第二次相遇。此时，统计出来的前进次数就是环长。

Question2：先做一个抽象推断，假设从链表头节点到入环节点的距离是D，从入环节点到两个指针首次相遇的距离是S1，从首次相遇点回到入环节点的距离是S2.

那么，当两个指针首次相遇时，各自所走的距离是多少呢？

指针p1一次只走1步，所走的距离是D+S1.<br>
指针p2一次走2步，多走了n（n>=1）整圈，所走的距离是D+S1+n(S1+S2).<br>
由于p2的速度是p1的2倍，所以相同时间所走的距离也是p1的2倍，因此：<br>
$2(D+S1)=D+S1+n(S1+S2)$ <br>
等式经过整理得出：<br>
$D=(n-1)S1+S2$ <br>

也就是说，从链表头节点到入环点的距离，等于从首次相遇点绕环n-1圈再回到入环点的距离。

这样一来，只要把其中一个指针放回到头节点的位置，另一个指针保持在首次相遇的点，两个指针都是每次向前走1步。那么，它们最终相遇的节点，就是入环节点。


## 5.3 最小栈的实现

**题目**：实现一个栈，该栈带有出栈(pop)、入栈(push)、取最小元素(get_min)3个方法。要保证这3个方法的时间复杂度都是$O(1)$.

***注意***：只暂存一个最小值是不够的，我们需要存储栈中曾经的最小值，作为“备胎”。_

***详细解法***：

1. 设原有的栈叫做栈A，此时创建一个额外的“备胎”栈B，用于辅助栈A。<br>
2. 当第1个元素进入栈A时，让新元素也进入栈B。这个唯一的元素是栈A的当前最小值。<br>
3. 之后，每当新元素进入栈A时，比较新元素和栈A当前最小值的大小，如果小于栈A当前的最小值，则让新元素进入栈B，此时栈B的栈顶元素就是栈A当前最小值。<br>
4. 每当栈A有元素出栈时，如果出栈元素是栈A当前最小值，则让栈B的栈顶元素也出栈。此时栈B余下的栈顶元素所指向的，是栈A中原本第2小的元素，代替刚才的出栈元素成为栈A的当前最小值。（备胎转正）<br>
5. 当调用get_min方法时，返回栈B的栈顶所存储的值，这也是栈A的最小值。

显然，该解法中进栈、出栈、取最小值的时间复杂度都是$O(1)$，最坏情况空间复杂度是$O(n)$.



In [2]:
# 代码实现

class MinStack:
    def __init__(self):
        self.main_stack = []
        self.min_stack = []

    def push(self, element):
        self.main_stack.append(element)
        # 若辅助栈为空，或新元素的值小于或等于辅助栈栈顶元素的值
        # 将新元素压入辅助栈
        if len(self.min_stack) == 0 or element <= self.min_stack[len(self.min_stack)-1]:
            self.min_stack.append(element)

    def pop(self):
        # 如果出栈元素和辅助栈栈顶元素的值相等，辅助栈的栈顶元素出栈
        if self.main_stack[len(self.main_stack)-1] == self.min_stack[len(self.min_stack)-1]:
            self.min_stack.pop()
        return self.main_stack.pop()

    def get_min(self):
        if len(self.main_stack) == 0:
            return None
        return self.min_stack[len(self.min_stack)-1]

my_stack = MinStack()
my_stack.push(4)
my_stack.push(9)
my_stack.push(7)
my_stack.push(3)
my_stack.push(8)
my_stack.push(5)
print(my_stack.get_min())
my_stack.pop()
my_stack.pop()
my_stack.pop()
print(my_stack.get_min())

3
4


## 5.4 如何求出最大公约数

**题目**：写一段代码，求出两个整数的最大公约数，要尽量优化算法的性能。


In [5]:
# 简单思路，暴力枚举
"""
从较小整数的一半开始，试图找到一个合适的整数i，看看这个整数能否被a和b同时整除。
"""

def get_greatest_common_divisor(a, b):
    big = max(a, b)
    small = min(a, b)
    if big % small == 0:
        return small
    for i in range(small//2, 1, -1):
        if small % i == 0 and big % i == 0:
            return i 
    return 1

print(get_greatest_common_divisor(25, 5))
print(get_greatest_common_divisor(100, 75))
print(get_greatest_common_divisor(99, 55))

5
25
11


**上述方法存在的问题**：假设输入的整数是10000和10001，那么该方法就要循环$10000/2-1=4999$次！

***解题思路***：辗转相除法，又名欧几里得算法(Euclidean algorithm),目的是求出两个正整数的最大公约数。该算法基于一个定理**两个正整数a和b(a>b)，它们的最大公约数等于a除以b的余数c和b之间的最大公约数。**

首先，计算出a除以b的余数c，把问题转化成求b和c的最大公约数；然后计算出b除以c的余数d，把问题转化成求c和d的最大公约数；再计算c除以d的余数e，直到两个数可以整除，或者其中一个数减小到1为止。



In [6]:
# 辗转相除法的实现

def get_greatest_common_divisor_v2(a, b):
    big = max(a, b)
    small = min(a, b)
    if big % small == 0:
        return small
    return get_greatest_common_divisor_v2(big % small, small)

print(get_greatest_common_divisor_v2(25, 5))

5


单纯的辗转相除法存在的问题：当两个整数较大时，做a%b取模运算的性能会比较差。

更相减损术的原理：**两个正整数a和b(a>b)，它们的最大公约数等于a-b的差值c和较小数b的最大公约数。**

通过递归简化问题：首先，计算出a和b的差值c(假设a>b)，把问题转化成求b和c的最大公约数；然后计算出c和b的差值d(假设c>b)，把问题转化成求b和d的最大公约数；再计算出b和d的差值e(假设b>d)，把问题转化成求d和e的最大公约数...以此类推，逐渐把两个较大整数之间的运算简化成两个较小整数之间的运算，直到两个数相等为止，最大公约数就是最终相等的这两个数的值。

In [8]:
# 更相减损术的代码实现

def get_greatest_common_divisor_v3(a, b):
    if a == b:
        return a
    big = max(a, b)
    small = min(a, b)
    return get_greatest_common_divisor_v3(big-small, small)

print(get_greatest_common_divisor_v3(25, 5))

5


更相减损术的缺点:该算法不稳定,当两数相差悬殊时,如计算10000和1的最大公约数,就要递归10000次.

***最优方法***:把辗转相除法和更相减损术的优势结合起来,在更相减损术的基础上使用移位运算.向右移1位代表除以2,向左移1位代表乘以2.

对于给出的正整数a和b,可以得到如下结论:

当a和b均为偶数时,$gcd(a,b)=2*gcd(a/2,b/2)=2*gcd(a>>1,b>>1)$<br>
当a为偶数,b为奇数时,$gcd(a,b)=gcd(a/2,b)=gcd(a>>1,b)$<br>
当a为奇数,b为偶数时,$gcd(a,b)=gcd(a,b/2)=gcd(a,b>>1)$<br>
当a和b均为奇数时,先利用更相减损术运算一次,$gcd(a,b)=gcd(b,a-b)$,此时a-b必然是偶数,然后又可以继续进行移位运算.



In [10]:
a = 2**5
print(a>>1)
print(a<<1)

16
64


In [1]:
# 最优方法的代码实现:

def get_greatest_common_divisor_v4(a, b):
    if a == b:
        return a
    # 判断奇偶的方式:让整数和1进行与运算,如果(a & 1) == 0,则说明a为偶数
    if (a & 1) == 0 and (b & 1) == 0:
        return get_greatest_common_divisor_v4(a>>1, b>>1) << 1
    elif (a & 1) == 0 and (b & 1) != 0:
        return get_greatest_common_divisor_v4(a>>1, b)
    elif (a & 1) != 0 and (b & 1) == 0:
        return get_greatest_common_divisor_v4(a, b>>1)
    else:
        big = max(a, b)
        small = min(a, b)
        return get_greatest_common_divisor_v4(big-small, small)

print(get_greatest_common_divisor_v4(25, 5))

5


上述解法的时间复杂度:

1. **暴力枚举法**:时间复杂度$O(min(a,b))$<br>
2. **辗转相除法**:近似为$O(log(max(a,b))),但是取模运算性能较差$<br>
3. **更相减损术**:避免了取模运算,算法性能不稳定,最坏时间复杂度为$O(max(a,b))$<br>
4. **更相减损术与移位相结合**:避免了取模运算,算法性能稳定,时间复杂度为$O(log(max(a,b)))$


## 5.5 如何判断一个数是否为2的整数次幂

